# Transformateurs Hugging Face

Hugging Face est une organisation qui s'efforce de résoudre et de démocratiser l'IA par le biais du langage naturel. Leur bibliothèque open-source 'transformers' est très populaire parmi la communauté NLP. Elle est très utile et puissante pour plusieurs tâches NLP et NLU. Elle comprend des milliers de modèles pré-entraînés dans plus de 100 langues. L'un des nombreux avantages de la bibliothèque transformers est qu'elle est compatible avec PyTorch et TensorFlow.

Nous pouvons installer les transformateurs directement à l'aide de pip comme indiqué ci-dessous :

In [ ]:
%%capture
!pip install transformers==3.5.1

Comme nous pouvons le voir, dans ce livre, nous utilisons la version 3.5.1 de Transformers. Maintenant que nous avons installé les transformateurs, commençons.

# Génération de l'intégration de BERT
Dans cette section, nous allons apprendre à extraire des embeddings à partir du BERT pré-entraîné. Considérons la phrase "J'aime Paris". Voyons comment obtenir l'encastrement contextualisé de tous les mots de la phrase en utilisant le modèle BERT pré-entraîné avec la bibliothèque de transformateurs de Hugging Face.

Tout d'abord, importons les modules nécessaires :

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

Ensuite, nous téléchargeons le modèle BERT pré-entraîné. Nous pouvons vérifier tous les modèles BERT pré-entraînés disponibles ici - https://huggingface.co/transformers/pre-trained_models.html.We utiliser le modèle 'bert-base-uncased'. Comme son nom l'indique, il s'agit du modèle BERT-base avec 12 encodeurs et il est entraîné avec des mots non accentués. Puisque nous utilisons la base BERT, la taille de la représentation sera de 768.

Téléchargez et chargez le modèle pré-entraîné bert-base-uncasedmodel :

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

Ensuite, nous téléchargeons et chargeons le tokenizer qui est utilisé pour le pré-entraînement du modèle sans base de Bert :

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Voyons maintenant comment prétraiter l'entrée avant de l'envoyer au BERT.

## Prétraitement de l'entrée
Définir la phrase :

In [ ]:
sentence = 'I love Paris'

Tokeniser la phrase et obtenir les tokens :

In [ ]:
tokens = tokenizer.tokenize(sentence)

Imprimons les jetons :

In [ ]:
print(tokens)

['i', 'love', 'paris']


Nous allons maintenant ajouter le jeton [CLS] au début et le jeton [SEP] à la fin de la liste des jetons :

In [ ]:
tokens = ['[CLS]'] + tokens + ['[SEP]']

Voyons la liste des jetons mise à jour :

In [ ]:
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]']


Comme nous pouvons le constater, nous avons le jeton [CLS] au début et le jeton sep à la fin de notre liste de jetons. Nous pouvons également observer que la longueur de nos jetons est de 5.

Supposons que nous devions maintenir la longueur de notre liste de jetons à 7. Dans ce cas, nous ajouterons deux jetons [PAD] à la fin de la liste, comme indiqué ci-dessous :

In [ ]:
tokens = tokens + ['[PAD]'] + ['[PAD]']

Imprimons notre liste de jetons mise à jour :

In [ ]:
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']


Comme nous pouvons le constater, la liste de jetons est maintenant composée de jetons [PAD] et la longueur de notre liste de jetons est de 7.

Ensuite, nous créons le masque d'attention. Nous fixons la valeur du masque d'attention à 1 si le jeton n'est pas un jeton [PAD], sinon nous fixons la valeur du masque d'attention à 0, comme indiqué ci-dessous :

In [ ]:
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]

Imprimons le masque d'attention :

In [ ]:
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


Comme nous pouvons le constater, les valeurs du masque d'attention sont 0 à la position où se trouve le jeton [PAD] et 1 aux autres positions.

Ensuite, nous convertissons tous les tokens en leurs token_ids, comme indiqué ci-dessous :

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)

Jetons un coup d'œil aux token_ids :

In [ ]:
print(token_ids)

[101, 1045, 2293, 3000, 102, 0, 0]


D'après la sortie ci-dessus, nous pouvons observer que chaque jeton est associé à un identifiant de jeton unique.

Nous convertissons maintenant les token_ids et les attention_mask en tenseurs, comme indiqué ci-dessous :

In [ ]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

C'est tout. Ensuite, nous transmettons les token_ids et attention_mask au modèle BERT pré-entraîné et obtenons l'intégration.

## Obtenir l'intégration

Comme le montre le code suivant, nous transmettons les token_ids et les attention_mask au modèle et obtenons les embeddings. Notez que le modèle renvoie la sortie sous la forme d'un tuple avec deux valeurs. La première valeur indique la représentation de l'état caché, hidden_rep, et consiste en la représentation de tous les tokens obtenus à partir de l'encodeur final (encodeur 12), et la seconde valeur, cls_head, consiste en la représentation du token [CLS] :

In [ ]:
hidden_rep, cls_head = model(token_ids, attention_mask = attention_mask)

Dans le code précédent, hidden_rep contient l'intégration (représentation) de tous les tokens de notre entrée. Imprimons la forme du tenseur hidden_rep :

In [ ]:
print(hidden_rep.shape)


torch.Size([1, 7, 768])


La taille [1,7,768] indique le [batch_size, sequence_length, hidden_size].

Notre taille de lot est 1, la longueur de séquence est la longueur du jeton, puisque nous avons 7 jetons, la longueur de séquence est 7, et la taille cachée est la taille de représentation (embedding) et elle est 768 pour le modèle BERT-base.

Nous pouvons obtenir la représentation de chaque jeton comme suit :

- hidden_rep[0][0] donne la représentation du premier token qui est [CLS]
- hidden_rep[0][1] donne la représentation du deuxième jeton, qui est "I".
- hidden_repo[0][2] donne la représentation du troisième jeton qui est 'love'

De cette manière, nous pouvons obtenir la représentation contextuelle de tous les jetons. Il s'agit en fait de l'enchâssement contextualisé de tous les mots de la phrase donnée.

Jetons maintenant un coup d'œil sur le cls_head. Il contient la représentation du token [CLS]. Imprimons la forme de cls_head :

In [ ]:
print(cls_head.shape)

torch.Size([1, 768])


Le code ci-dessus s'imprimera :

torch.Size([1, 768])

La taille [1,768] indique le [batch_size, hidden_size].

Nous avons appris que cls_head contient la représentation agrégée de la phrase, nous pouvons donc utiliser cls_head comme représentation de la phrase donnée "J'aime Paris".

Nous avons appris à extraire des embeddings du BERT pré-entraîné. Mais il s'agit des encastrements obtenus uniquement à partir de la couche d'encodage la plus élevée du BERT, à savoir l'encodeur 12. Pouvons-nous également extraire les enchâssements de toutes les couches d'encodage du BERT ? Oui ! Nous verrons comment procéder dans la section suivante.